# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [5]:
# Load CSV
city_data = "../output_data/city_data.csv"
weather_data = pd.read_csv(city_data)
clean_weather_data = weather_data.dropna()
del clean_weather_data["Unnamed: 0"]
clean_weather_data.head(20)


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Punta Arenas,-53.15,-70.92,41.00,75,100,24.16
1,Kapaa,22.08,-159.32,84.47,58,40,14.99
2,Port Elizabeth,-33.92,25.57,59.00,100,90,13.87
3,Avarua,-21.21,-159.78,78.80,78,92,17.22
4,Saint-Philippe,-21.36,55.77,63.59,77,75,3.36
5,New Norfolk,-42.78,147.06,57.56,61,100,1.99
6,Rikitea,-23.12,-134.97,73.47,84,99,14.67
7,Katsuura,35.13,140.30,83.64,88,75,3.36
8,Lakes Entrance,-37.88,147.98,66.00,57,52,1.99
9,Albany,42.60,-73.97,71.78,78,14,8.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# Use lat/Long as locations and humidity as the weight
locations = clean_weather_data[["Latitude", "Longitude"]]
humidity = clean_weather_data["Humidity"].astype(float)

In [8]:
#Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Create perfect weather df
sunny = clean_weather_data[clean_weather_data["Cloudiness"] == 0]
# Temps above 70 degrees
sunny_warm = sunny[sunny["Temperature"] > 70]
# Temps also below 80 degrees
sunny_warm_cool = sunny_warm[sunny_warm["Temperature"] < 80]
# Wind speed below 10 mph.
sunny_warm_cool_calm = sunny_warm_cool[sunny_warm_cool["Wind Speed"] < 10]
# Humidity below 60%
sunny_warm_cool_calm_dry = sunny_warm_cool_calm[sunny_warm_cool_calm["Humidity"] < 60]

perfect_weather = sunny_warm_cool_calm_dry.reset_index()
del perfect_weather["index"]
perfect_weather

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Kabalo,-6.05,26.92,74.52,36,0,3.22
1,Najrān,17.49,44.13,77.00,29,0,2.19
2,Vanderhoof,54.02,-124.02,75.31,46,0,1.52
3,Blackfoot,43.19,-112.34,70.30,4,0,1.01
4,Turaif,31.67,38.66,77.00,16,0,9.17
5,Barra do Garças,-15.89,-52.26,79.50,22,0,1.28
6,Zhangye,38.93,100.45,71.65,35,0,2.10
7,Charters Towers,-20.10,146.27,76.33,35,0,8.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(perfect_weather["City"])):

    lat = perfect_weather.loc[city]["Latitude"]
    lng = perfect_weather.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotel_df.append(hotel_response["results"][0]["name"])
    except:
        hotel_df.append("Nearest hotel not found")

# Dataframe with nearest hotel
perfect_weather["Nearest Hotel"] = hotel_df
perfect_weather

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Nearest Hotel
0,Kabalo,-6.05,26.92,74.52,36,0,3.22,Nearest hotel not found
1,Najrān,17.49,44.13,77.00,29,0,2.19,Nearest hotel not found
2,Vanderhoof,54.02,-124.02,75.31,46,0,1.52,Nearest hotel not found
3,Blackfoot,43.19,-112.34,70.30,4,0,1.01,Nearest hotel not found
4,Turaif,31.67,38.66,77.00,16,0,9.17,Nearest hotel not found
5,Barra do Garças,-15.89,-52.26,79.50,22,0,1.28,Nearest hotel not found
6,Zhangye,38.93,100.45,71.65,35,0,2.10,Nearest hotel not found
7,Charters Towers,-20.10,146.27,76.33,35,0,8.93,Nearest hotel not found


I cannot get the map to work. I don't see where to get the country information from the API.

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_weather.iterrows()]
locations = perfect_weather[["Lat", "Lng"]]

KeyError: 'Country'